In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from fastdtw import fastdtw 
from scipy.spatial.distance import euclidean, cosine
import os

In [2]:
# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

I0000 00:00:1737877614.786737   29770 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Max


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737877614.864036   29910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737877614.872273   29923 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
import cv2
import mediapipe as mp
import pandas as pd

# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def extract_keypoints_to_dataframe(video_path):
    # OpenCV로 비디오 읽기
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 RGB 변환 (Mediapipe는 RGB 입력 필요)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Mediapipe Pose 추론
        result = pose.process(frame_rgb)

        if result.pose_world_landmarks:
            # 키포인트 저장용 딕셔너리
            keypoints = {"frame_index": frame_index}

            # 각 키포인트의 x, y, z, visibility 저장
            for i, landmark in enumerate(result.pose_world_landmarks.landmark):
                keypoints[f"x_{i}"] = landmark.x
                keypoints[f"y_{i}"] = landmark.y
                keypoints[f"z_{i}"] = landmark.z
                keypoints[f"visibility_{i}"] = landmark.visibility

            keypoints_list.append(keypoints)
        
        frame_index += 1

    cap.release()

    # Pandas DataFrame으로 변환
    df = pd.DataFrame(keypoints_list)
    return df


I0000 00:00:1737878147.153810   29770 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Max


W0000 00:00:1737878147.230525   37096 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737878147.239449   37104 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [6]:
# 비디오 경로
video_path = "video_chal.mp4"

# 데이터프레임으로 변환
df_keypoints = extract_keypoints_to_dataframe(video_path)

# 데이터프레임 저장
# df_keypoints.to_csv("keypoints.csv", index=False)
# print("Keypoints DataFrame 저장 완료!")

df_keypoints.head()

,frame_index,x_0,y_0,z_0,visibility_0,x_1,y_1,z_1,visibility_1,x_2,...,z_30,visibility_30,x_31,y_31,z_31,visibility_31,x_32,y_32,z_32,visibility_32
0,0,0.018019,-0.626386,0.182640,0.999864,-0.007780,-0.656338,0.158403,0.999861,-0.007221,...,0.067798,0.910501,0.007314,0.868515,-0.047795,0.928920,0.056749,0.879568,0.006675,0.875751
1,1,0.019939,-0.626173,0.199991,0.999864,-0.006196,-0.656009,0.175115,0.999862,-0.005715,...,0.047315,0.909093,0.007282,0.863207,-0.048087,0.926094,0.064224,0.872164,-0.008887,0.871847
2,2,0.021944,-0.622563,0.205574,0.999859,-0.004770,-0.652104,0.181406,0.999859,-0.004357,...,0.023660,0.907106,0.007158,0.860687,-0.050421,0.920945,0.069430,0.867075,-0.032090,0.865580
3,3,0.025728,-0.619344,0.206026,0.999853,-0.001258,-0.648881,0.182056,0.999856,-0.000903,...,0.018217,0.906540,0.007018,0.860444,-0.049797,0.916934,0.070104,0.866559,-0.037675,0.861521
4,4,0.027972,-0.618716,0.211207,0.999853,0.000834,-0.648220,0.187201,0.999857,0.001156,...,0.015944,0.906390,0.006490,0.859835,-0.050103,0.914114,0.069918,0.865937,-0.039200,0.858521
